In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/N_Graph

/content/drive/MyDrive/N_Graph


In [3]:
import pandas as pd
import json
import urllib
from tqdm import tqdm

# **JSON Loader**

In [ ]:
f = open('input.json',)

In [ ]:
data = json.load(f)

In [ ]:
f.close()

In [ ]:
nodes = []
for i in tqdm(data['Graph']):
    px = []
    px.append(i['id'])
    px.append(i['label'])
    nodes.append(px)
  
# Closing file
f.close()

In [ ]:
rels = []
for i in tqdm(data['Relationship']):
  px = []
  px.append(i['start'])
  px.append(i['end'])
  rels.append(px)
  
# Closing file
f.close()

In [ ]:
xd = pd.DataFrame(
            nodes, columns=['id', 'label'])
xr = pd.DataFrame(
            rels, columns=['start', 'end'])

In [ ]:
xd.head()

In [ ]:
xr.head()

In [ ]:
xr.tail()

In [ ]:
orr = xr.groupby(['end'])

# **Split into PRO CON**

In [ ]:
xd.id[xd.id == '74356'].index.tolist()

In [ ]:
xd.id[xd.id == '77483'].index.tolist()

In [ ]:
xd.id[xd.label == 'with discussion and facts on a particular view that they may hold for the'].index.tolist()

In [ ]:
split_point = 74356
in_split = 77586

In [ ]:
casualty = xd[:split_point]

In [ ]:
neutral = xd[split_point:in_split]

In [ ]:
pro_con = xd[in_split:]

In [ ]:
casualty.tail()

In [ ]:
neutral.head()

In [ ]:
neutral.tail()

In [ ]:
pro_con.head()

# **Fiass Implementation**

In [ ]:
!pip install faiss-gpu
!pip install sentence_transformers
!pip install --user -U nltk

In [ ]:
import numpy as np
import faiss
import requests
from io import StringIO
import pandas as pd
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from datetime import datetime
from sentence_transformers import SentenceTransformer
import os
import nltk
nltk.download("punkt")
nltk.download('stopwords')

In [ ]:
data = casualty
data.head()

In [ ]:
sentences = data['label'].tolist()
sentences[:10]

In [ ]:
sentences_id = data['id'].tolist()
sentences_id[:10]

In [ ]:
sentences = [
    sentence.replace('\n', '') for sentence in list(set(sentences)) if type(sentence) is str
    ]

In [ ]:
with open('backup_sentences.txt', 'w') as fp:
    fp.write('\n'.join(sentences))

In [ ]:
import torch

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens', device='cuda').to(device)

sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

In [ ]:
sentence_embeddings.shape[0]

In [ ]:
with open(f'./sim_sentences/embeddings_X.npy', 'wb') as fp:
    np.save(fp, sentence_embeddings[0:256])

In [ ]:
split = 256
file_count = 0
for i in range(0, sentence_embeddings.shape[0], split):
    end = i + split
    if end > sentence_embeddings.shape[0] + 1:
        end = sentence_embeddings.shape[0] + 1
    file_count = '0' + str(file_count) if file_count < 0 else str(file_count)
    with open(f'./sim_sentences/embeddings_{file_count}.npy', 'wb') as fp:
        np.save(fp, sentence_embeddings[i:end, :])
    print(f"embeddings_{file_count}.npy | {i} -> {end}")
    file_count = int(file_count) + 1

In [ ]:
d = sentence_embeddings.shape[1]
d

In [ ]:
nlist = 50
quantizer = faiss.IndexFlatL2(d)

In [ ]:
res = faiss.StandardGpuResources()

In [ ]:
f_index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

In [ ]:
index = faiss.index_cpu_to_gpu(res, 0, f_index)

In [ ]:
index.is_trained

In [ ]:
assert not index.is_trained

In [ ]:
index.train(sentence_embeddings)
index.is_trained

In [ ]:
index.add(sentence_embeddings)
index.ntotal

In [ ]:
k = 1
xq = model.encode(["O Brasil ser/u00e1 beneficiado com a privatiza"])
index.nprobe = 10

In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)

In [ ]:
[f'{i}: {sentences[i]}' for i in I[0]]

In [ ]:
mapp = I[0][0]
print(I[0][0])

In [ ]:
sentences[I[0][0]]

In [ ]:
data.id[data.label == 'p brasiliensi'].index.tolist()[0]

In [ ]:
datad = pro_con
datad.head()

In [ ]:
sentence_b = datad['label'].tolist()
len(sentence_b)

In [ ]:
sentence_b_id = datad['id'].tolist()
len(sentence_b_id)

# **Cluster Gun**

In [ ]:
from IPython.display import clear_output

In [ ]:
from tqdm import tqdm

In [ ]:
sentence_b[:5]

In [ ]:
sentence_b_id[:5]

In [ ]:
brels = []

In [ ]:
brels

In [ ]:
for x in tqdm(range(len(sentence_b))):
  k = 4
  xq = model.encode([sentence_b[x]])
  D, I = index.search(xq, k)
  X = sentences[I[0][0]]
  Y = sentence_b[x]
  px = []
  px.append(casualty.id[casualty.label == X].index.tolist()[0])
  px.append(str(sentence_b_id[x]))
  brels.append(px)

In [ ]:
nxr = pd.DataFrame(
            brels, columns=['start', 'end'])
nxr.to_csv('./new_rels.csv', index=False)

In [ ]:
len(brels)

# **Strict Work of assigning parents**

In [ ]:
gf = pd.read_csv('./new_rels.csv')
gf.head()

In [ ]:
nst = gf['start'].tolist()
nen = gf['end'].tolist()

In [ ]:
for x in tqdm(range(len(nst))):
  if str(nst[x]) in orr.groups.keys():
    dfd = orr.get_group(str(nst[x]))
    imp = dfd['start'].tolist()
    for z in range(len(imp)):
      px = []
      px.append(imp[z])
      px.append(nen[x])
      rels.append(px)

In [ ]:
xnr = pd.DataFrame(
            rels, columns=['start', 'end'])

In [ ]:
xnr.to_csv('./new_optic_parent_rels.csv', index=False)

In [ ]:
rels[2023920:]

In [ ]:
len(rels)

# **Merging Graphs**

In [ ]:
xnr = pd.read_csv('./new_optic_parent_rels.csv')

In [ ]:
lab = xd['label'].tolist()
xd['concept'] = lab
xd['type'] = 'node'
xd.head()

In [ ]:
xnr['type'] = 'relationship'
xnr.head()

In [ ]:
len(xnr)

In [ ]:
xnr.drop_duplicates(keep='first', inplace=True)
len(xnr)

In [ ]:
L1 = [{k: v for k, v in x.items() if pd.notnull(v)} for x in xd.to_dict('r')]
L2 = [{k: v for k, v in x.items() if pd.notnull(v)} for x in xnr.to_dict('r')]

In [ ]:
with open('./Graph.json', 'w') as file:
    json.dump({ "Graph": L1, "Relationship": L2}, file)

# **New JSON Loader**

In [ ]:
f = open('./Graph.json',)

In [ ]:
data = json.load(f)

In [ ]:
f.close()

In [ ]:
nodes = []
for i in tqdm(data['Graph']):
    px = []
    px.append(int(i['id']))
    px.append(i['label'])
    nodes.append(px)
  
# Closing file
f.close()

In [ ]:
rels = []
for i in tqdm(data['Relationship']):
  px = []
  px.append(int(i['start']))
  px.append(int(i['end']))
  rels.append(px)
  
# Closing file
f.close()

In [ ]:
xd = pd.DataFrame(
            nodes, columns=['id', 'label'])
xr = pd.DataFrame(
            rels, columns=['start', 'end'])

In [ ]:
xd.head()

In [ ]:
len(xd)

In [ ]:
xd = xd.drop_duplicates(subset=['id'], keep='first', ignore_index=True)

In [ ]:
len(xd)

In [ ]:
xr.head()

In [ ]:
xr.tail()

In [ ]:
lab = xd['label'].tolist()
xd['concept'] = lab
xd['type'] = 'node'
xd.tail()

In [ ]:
xr['type'] = 'relationship'
xr.head()

In [ ]:
chunky = []
for i in range(0, 338000):
  chunky.append(i)

In [ ]:
chunky[:5]

In [ ]:
abs = xd['id'].tolist()
abs[:5]

In [ ]:
a = list(set(abs)-set(chunky))

In [ ]:
len(a)

In [ ]:
a.sort()

In [ ]:
a[:5]

In [ ]:
len(xr)

In [ ]:
xr = xr[~xr['start'].isin(a)]
xr = xr[~xr['end'].isin(a)]

In [ ]:
len(xr)

In [ ]:
xr.head()

In [ ]:
L1 = [{k: v for k, v in x.items() if pd.notnull(v)} for x in xd.to_dict('r')]
L2 = [{k: v for k, v in x.items() if pd.notnull(v)} for x in xr.to_dict('r')]

In [ ]:
with open('./Graph.json', 'w') as file:
    json.dump({ "Graph": L1, "Relationship": L2}, file)

# **Re-Indexing**

In [ ]:
ff = pd.read_csv('./Indexing.csv')

In [ ]:
actual = ff['a'].tolist()
graph = ff['g'].tolist()

In [ ]:
new = []
for i in tqdm(range(len(actual))):
  flag = 0
  for j in range(len(graph)):
    if int(actual[i]) == int(graph[j]):
      graph.remove(graph[j])
      flag = 1
      break
  if flag == 1:
    new.append(int(actual[i]))
  else:
    new.append(-1)

In [ ]:
dataf = []
for i in tqdm(range(len(actual))):
  px = []
  px.append(int(actual[i]))
  px.append(new[i])
  dataf.append(px)

In [ ]:
xx = pd.DataFrame(
            dataf, columns=['actual', 'original'])

In [ ]:
xx.to_csv('./RmeIndex.csv')

# **Applying new indexes**

In [ ]:
f = open('./Graph.json',)

In [ ]:
data = json.load(f)

In [ ]:
f.close()

In [ ]:
nodes = []
for i in tqdm(data['Graph']):
    px = []
    px.append(int(i['id']))
    px.append(i['label'])
    nodes.append(px)
  
# Closing file
f.close()

In [ ]:
rels = []
for i in tqdm(data['Relationship']):
  px = []
  px.append(int(i['start']))
  px.append(int(i['end']))
  rels.append(px)
  
# Closing file
f.close()

In [ ]:
xd = pd.DataFrame(
            nodes, columns=['id', 'label'])
xr = pd.DataFrame(
            rels, columns=['start', 'end'])

In [ ]:
xd.head()

In [ ]:
xr.head()

In [ ]:
nin = xd['id'].tolist()
nsti = xr['start'].tolist()
neni = xr['end'].tolist()

In [ ]:
indexes = pd.read_csv('./Rme.csv')
indexes.head()

In [ ]:
org = indexes['original'].tolist()
n_in = indexes['new'].tolist()

In [ ]:
nid = []
for x in tqdm(nin):
  nid.append(int(n_in[x]))

In [ ]:
xd['id'] = nid
xd.tail()

In [ ]:
xd = xd.sort_values('id')
xd = xd.reset_index(drop=True)
xd.tail()

In [ ]:
d = []
r = []

In [ ]:
for x in tqdm(range(len(nsti))):
  d.append(int(n_in[nsti[x]]))
  r.append(int(n_in[neni[x]]))

In [ ]:
xr['start'] = d
xr['end'] = r

In [ ]:
lab = xd['label'].tolist()
xd['concept'] = lab
xd['type'] = 'node'
xd.tail()

In [ ]:
xr['type'] = 'relationship'
xr.head()

In [ ]:
L1 = [{k: v for k, v in x.items() if pd.notnull(v)} for x in xd.to_dict('r')]
L2 = [{k: v for k, v in x.items() if pd.notnull(v)} for x in xr.to_dict('r')]

In [ ]:
with open('./Re-Graph.json', 'w') as file:
    json.dump({ "Graph": L1, "Relationship": L2}, file)

# **Assigning Effects to the relations**

In [ ]:
f = open('./Re-Graph.json',)

In [ ]:
data = json.load(f)

In [ ]:
f.close()

In [ ]:
nodes = []
for i in tqdm(data['Graph']):
    px = []
    px.append(int(i['id']))
    px.append(i['label'])
    px.append(i['concept'])
    px.append(i['type'])
    nodes.append(px)
  
# Closing file
f.close()

100%|██████████| 333075/333075 [00:00<00:00, 461524.57it/s]


In [ ]:
rels = []
for i in tqdm(data['Relationship']):
  px = []
  px.append(int(i['start']))
  px.append(int(i['end']))
  px.append(i['effect'])
  px.append(i['type'])
  rels.append(px)
  
# Closing file
f.close()

100%|██████████| 2023904/2023904 [00:03<00:00, 638027.85it/s]


In [ ]:
xd = pd.DataFrame(
            nodes, columns=['id', 'label', 'concept', 'type'])
xr = pd.DataFrame(
            rels, columns=['start', 'end', 'effect', 'type'])

In [ ]:
lab = xd['label'].tolist()
xd['concept'] = lab
xd['type'] = 'node'
xd.tail()

,id,label,concept,type
333070,333070,valuable neighborhood school,valuable neighborhood school,node
333071,333071,teacher independence,teacher independence,node
333072,333072,teacher motivation,teacher motivation,node
333073,333073,innovation making the profession attractive,innovation making the profession attractive,node
333074,333074,charter school choice,charter school choice,node


In [ ]:
xr['type'] = 'relationship'
xr.head()

,start,end,type
0,0,1,relationship
1,0,10,relationship
2,0,89,relationship
3,0,53,relationship
4,0,55,relationship


In [ ]:
xd.head()

,id,label,concept,type
0,0,accident,accident,node
1,1,death,death,node
2,2,disease,disease,node
3,3,pneumonium,pneumonium,node
4,4,cancer,cancer,node


In [ ]:
xr.head()

,start,end,type
0,0,1,relationship
1,0,10,relationship
2,0,89,relationship
3,0,53,relationship
4,0,55,relationship


**Implementing VADER**

In [ ]:
!pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 15.1 MB/s 


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
su = SentimentIntensityAnalyzer()

In [ ]:
nin = xd['id'].tolist()
nlab = xd['label'].tolist()
nsti = xr['start'].tolist()
neni = xr['end'].tolist()

In [ ]:
effect = []
for x in tqdm(range(len(nsti))):
  premise = nlab[nsti[x]]
  hypothesis = nlab[neni[x]]
  data = su.polarity_scores(f'{premise} but {hypothesis}')
  if data['compound'] <= 0:
    effect.append('negative')
  else:
    effect.append('positive')

  0%|          | 0/2023904 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2344: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  FutureWarning,
  1%|          | 15673/2023904 [1:30:00<185:12:07,  3.01it/s]

In [ ]:
xr['effect'] = effect
xr.head()

,start,end,type,effect
0,0,1,relationship,negative
1,0,10,relationship,negative
2,0,89,relationship,negative
3,0,53,relationship,negative
4,0,55,relationship,negative


In [ ]:
L1 = [{k: v for k, v in x.items() if pd.notnull(v)} for x in xd.to_dict('r')]
L2 = [{k: v for k, v in x.items() if pd.notnull(v)} for x in xr.to_dict('r')]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  


In [ ]:
with open('./ReNewEf-Graph.json', 'w') as file:
    json.dump({ "Graph": L1, "Relationship": L2}, file)

# **Adding Arg Graph**

In [ ]:
xd.tail()

,id,label,concept,type
333070,333070,valuable neighborhood school,valuable neighborhood school,node
333071,333071,teacher independence,teacher independence,node
333072,333072,teacher motivation,teacher motivation,node
333073,333073,innovation making the profession attractive,innovation making the profession attractive,node
333074,333074,charter school choice,charter school choice,node


In [ ]:
thr = len(xd)
nds = pd.read_csv('./arg_G.csv')
rsd = pd.read_csv('./arg_graph.csv') 

In [ ]:
nds.head()

,id,label,concept
0,0,difficulty in defining fair use,difficulty in defining fair use
1,1,court battle,court battle
2,2,court,court
3,3,company,company
4,4,user,user


In [ ]:
idf = nds['id'].tolist()
noid = []
for x in idf:
  noid.append(x + thr)
nds['id'] = noid

In [ ]:
rsd.head()

,start,end,effect
0,0,1,positive
1,1,2,negative
2,1,3,negative
3,1,4,negative
4,5,6,positive


In [ ]:
sti = rsd['start'].tolist()
eni = rsd['end'].tolist()
sd = []
se = []
for x in range(len(sti)):
  sd.append(sti[x] + thr)
  se.append(eni[x] + thr)
rsd['start'] = sd
rsd['end'] = se

In [ ]:
rsd['type'] = 'relationship'
nds['type'] = 'node'

In [ ]:
rsd.head()

,start,end,effect,type
0,333075,333076,positive,relationship
1,333076,333077,negative,relationship
2,333076,333078,negative,relationship
3,333076,333079,negative,relationship
4,333080,333081,positive,relationship


In [ ]:
nds.head()

,id,label,concept,type
0,333075,difficulty in defining fair use,difficulty in defining fair use,node
1,333076,court battle,court battle,node
2,333077,court,court,node
3,333078,company,company,node
4,333079,user,user,node


In [ ]:
xd = xd.append(nds)
xr = xr.append(rsd)

In [ ]:
xd = xd.reset_index(drop=True)
xr = xr.reset_index(drop=True)

In [ ]:
xd.tail()

,id,label,concept,type
337677,337677,employer to campaign year round,employer to campaign year round,node
337678,337678,collapsing of the skull of the partially born ...,collapsing of the skull of the partially born ...,node
337679,337679,risk of damage to the woman,risk of damage to the woman,node
337680,337680,random sobriety stop,random sobriety stop,node
337681,337681,unjust detection of other crime,unjust detection of other crime,node


In [ ]:
xr.tail()

,start,end,type,effect
2032999,337668,337669,relationship,negative
2033000,337674,337675,relationship,positive
2033001,337678,335675,relationship,negative
2033002,337678,337679,relationship,negative
2033003,337680,337681,relationship,positive


In [ ]:
L1 = [{k: v for k, v in x.items() if pd.notnull(v)} for x in xd.to_dict('r')]
L2 = [{k: v for k, v in x.items() if pd.notnull(v)} for x in xr.to_dict('r')]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  


In [ ]:
with open('./ReNewEfArg-Graph.json', 'w') as file:
    json.dump({ "Graph": L1, "Relationship": L2}, file)

# **Assigning & Calculating Ranks**

In [ ]:
f = open('./ReNewEfArg-Graph.json',)

In [ ]:
data = json.load(f)

In [ ]:
f.close()

In [ ]:
nodes = []
for i in tqdm(data['Graph']):
    px = []
    px.append(int(i['id']))
    px.append(i['label'])
    px.append(i['concept'])
    px.append(i['type'])
    nodes.append(px)
  
# Closing file
f.close()

100%|██████████| 337682/337682 [00:00<00:00, 581402.38it/s]


In [ ]:
rels = []
for i in tqdm(data['Relationship']):
  px = []
  px.append(int(i['start']))
  px.append(int(i['end']))
  px.append(i['effect'])
  px.append(i['type'])
  rels.append(px)
  
# Closing file
f.close()

100%|██████████| 2033004/2033004 [00:04<00:00, 501845.93it/s]


In [ ]:
xd = pd.DataFrame(
            nodes, columns=['id', 'label', 'concept', 'type'])
xr = pd.DataFrame(
            rels, columns=['start', 'end', 'effect', 'type'])

In [ ]:
xd.head()

,id,label,concept,type
0,0,accident,accident,node
1,1,death,death,node
2,2,disease,disease,node
3,3,pneumonium,pneumonium,node
4,4,cancer,cancer,node


In [ ]:
xr.head()

,start,end,effect,type
0,0,1,negative,relationship
1,0,10,negative,relationship
2,0,89,negative,relationship
3,0,53,negative,relationship
4,0,55,negative,relationship


In [ ]:
ndt = xd['id'].tolist()
eng = xr.groupby(['end'])
sng = xr.groupby(['start'])

In [ ]:
k = 10000
n = len(xd)

In [ ]:
im = k/n

In [ ]:
im

0.02961366018917206

In [ ]:
ranks = []

In [ ]:
for x in tqdm(ndt):
  r = 0
  if x in eng.groups.keys():
    df = eng.get_group(x)
    ic = df['start'].tolist()
    for y in ic:
      if y in sng.groups.keys():
        dm = sng.get_group(y)
        l = len(dm)
        p = (k/l)*im
        r+=p
  ranks.append(r)

100%|██████████| 337682/337682 [08:19<00:00, 675.41it/s] 


In [ ]:
xd['ranks'] = ranks

In [ ]:
xd.head()

,id,label,concept,type,ranks
0,0,accident,accident,node,17399.134735
1,1,death,death,node,23440.786330
2,2,disease,disease,node,26235.653549
3,3,pneumonium,pneumonium,node,6641.914576
4,4,cancer,cancer,node,20368.378862


In [ ]:
xd.tail()

,id,label,concept,type,ranks
337677,337677,employer to campaign year round,employer to campaign year round,node,49.356100
337678,337678,collapsing of the skull of the partially born ...,collapsing of the skull of the partially born ...,node,0.000000
337679,337679,risk of damage to the woman,risk of damage to the woman,node,148.068301
337680,337680,random sobriety stop,random sobriety stop,node,0.000000
337681,337681,unjust detection of other crime,unjust detection of other crime,node,296.136602


In [ ]:
L1 = [{k: v for k, v in x.items() if pd.notnull(v)} for x in xd.to_dict('r')]
L2 = [{k: v for k, v in x.items() if pd.notnull(v)} for x in xr.to_dict('r')]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  


In [ ]:
with open('./ReNewEfArgRanks-Graph.json', 'w') as file:
    json.dump({ "Graph": L1, "Relationship": L2}, file)

# **Assigning Pos Rank**

In [4]:
f = open('./ReNewEfArgRanks-Graph.json',)

In [5]:
data = json.load(f)

In [6]:
f.close()

In [7]:
nodes = []
for i in tqdm(data['Graph']):
    px = []
    px.append(int(i['id']))
    px.append(i['label'])
    px.append(i['concept'])
    px.append(i['type'])
    px.append(i['ranks'])
    nodes.append(px)
  
# Closing file
f.close()

100%|██████████| 337682/337682 [00:00<00:00, 374481.86it/s]


In [8]:
rels = []
for i in tqdm(data['Relationship']):
  px = []
  px.append(int(i['start']))
  px.append(int(i['end']))
  px.append(i['effect'])
  px.append(i['type'])
  rels.append(px)
  
# Closing file
f.close()

100%|██████████| 2033004/2033004 [00:03<00:00, 530249.98it/s]


In [10]:
xd = pd.DataFrame(
            nodes, columns=['id', 'label', 'concept', 'type', 'ranks'])
xr = pd.DataFrame(
            rels, columns=['start', 'end', 'effect', 'type'])

In [11]:
xd.head()

,id,label,concept,type,ranks
0,0,accident,accident,node,17399.134735
1,1,death,death,node,23440.786330
2,2,disease,disease,node,26235.653549
3,3,pneumonium,pneumonium,node,6641.914576
4,4,cancer,cancer,node,20368.378862


In [12]:
xr.head()

,start,end,effect,type
0,0,1,negative,relationship
1,0,10,negative,relationship
2,0,89,negative,relationship
3,0,53,negative,relationship
4,0,55,negative,relationship


In [13]:
orr = xr.groupby(['effect'])

In [14]:
pos = orr.get_group('positive')
neg = orr.get_group('negative')

In [35]:
len(pos)

331787

In [36]:
len(neg)

1701217

In [15]:
ndt = xd['id'].tolist()
eng = pos.groupby(['end'])
sng = pos.groupby(['start'])

In [16]:
k = 10000
n = len(xd)

In [17]:
im = k/n

In [18]:
im

0.02961366018917206

In [ ]:
ranks = []

In [20]:
for x in tqdm(ndt):
  r = 0
  if x in eng.groups.keys():
    df = eng.get_group(x)
    ic = df['start'].tolist()
    for y in ic:
      if y in sng.groups.keys():
        dm = sng.get_group(y)
        l = len(dm)
        p = (k/l)*im
        r+=p
  ranks.append(r)

100%|██████████| 337682/337682 [01:27<00:00, 3878.09it/s]


In [21]:
xd['Pos'] = ranks

In [22]:
xd.head()

,id,label,concept,type,ranks,Pos
0,0,accident,accident,node,17399.134735,0.000000
1,1,death,death,node,23440.786330,13.324650
2,2,disease,disease,node,26235.653549,2072.677657
3,3,pneumonium,pneumonium,node,6641.914576,273.826409
4,4,cancer,cancer,node,20368.378862,0.000000


In [23]:
xd.tail()

,id,label,concept,type,ranks,Pos
337677,337677,employer to campaign year round,employer to campaign year round,node,49.356100,49.356100
337678,337678,collapsing of the skull of the partially born ...,collapsing of the skull of the partially born ...,node,0.000000,0.000000
337679,337679,risk of damage to the woman,risk of damage to the woman,node,148.068301,0.000000
337680,337680,random sobriety stop,random sobriety stop,node,0.000000,0.000000
337681,337681,unjust detection of other crime,unjust detection of other crime,node,296.136602,296.136602


In [24]:
ndt = xd['id'].tolist()
eng = neg.groupby(['end'])
sng = neg.groupby(['start'])

In [25]:
k = 10000
n = len(xd)

In [26]:
im = k/n

In [27]:
im

0.02961366018917206

In [28]:
ranks = []

In [29]:
for x in tqdm(ndt):
  r = 0
  if x in eng.groups.keys():
    df = eng.get_group(x)
    ic = df['start'].tolist()
    for y in ic:
      if y in sng.groups.keys():
        dm = sng.get_group(y)
        l = len(dm)
        p = (k/l)*im
        r+=p
  ranks.append(r)

100%|██████████| 337682/337682 [07:07<00:00, 789.73it/s] 


In [30]:
xd['Neg'] = ranks

In [31]:
xd.head()

,id,label,concept,type,ranks,Pos,Neg
0,0,accident,accident,node,17399.134735,0.000000,17508.337566
1,1,death,death,node,23440.786330,13.324650,23751.858104
2,2,disease,disease,node,26235.653549,2072.677657,24777.582845
3,3,pneumonium,pneumonium,node,6641.914576,273.826409,6487.896418
4,4,cancer,cancer,node,20368.378862,0.000000,20963.631060


In [32]:
xd.tail()

,id,label,concept,type,ranks,Pos,Neg
337677,337677,employer to campaign year round,employer to campaign year round,node,49.356100,49.356100,0.000000
337678,337678,collapsing of the skull of the partially born ...,collapsing of the skull of the partially born ...,node,0.000000,0.000000,0.000000
337679,337679,risk of damage to the woman,risk of damage to the woman,node,148.068301,0.000000,148.068301
337680,337680,random sobriety stop,random sobriety stop,node,0.000000,0.000000,0.000000
337681,337681,unjust detection of other crime,unjust detection of other crime,node,296.136602,296.136602,0.000000


In [33]:
L1 = [{k: v for k, v in x.items() if pd.notnull(v)} for x in xd.to_dict('r')]
L2 = [{k: v for k, v in x.items() if pd.notnull(v)} for x in xr.to_dict('r')]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  


In [34]:
with open('./ReNewEfArgRanksPN-Graph.json', 'w') as file:
    json.dump({ "Graph": L1, "Relationship": L2}, file)

# **Real Page Rank algorithm**

In [43]:
f = open('./ReNewEfArgRanks-Graph.json',)

In [44]:
data = json.load(f)

In [45]:
f.close()

In [46]:
nodes = []
for i in tqdm(data['Graph']):
    px = []
    px.append(int(i['id']))
    px.append(i['label'])
    px.append(i['concept'])
    px.append(i['type'])
    px.append(i['ranks'])
    nodes.append(px)
  
# Closing file
f.close()

100%|██████████| 337682/337682 [00:00<00:00, 476766.74it/s]


In [ ]:
rels = []
for i in tqdm(data['Relationship']):
  px = []
  px.append(int(i['start']))
  px.append(int(i['end']))
  px.append(i['effect'])
  px.append(i['type'])
  rels.append(px)
  
# Closing file
f.close()

100%|██████████| 2033004/2033004 [00:04<00:00, 449614.77it/s]


In [ ]:
xd = pd.DataFrame(
            nodes, columns=['id', 'label', 'concept', 'type', 'ranks'])
xr = pd.DataFrame(
            rels, columns=['start', 'end', 'effect', 'type'])

In [ ]:
xd.head()

,id,label,concept,type,ranks
0,0,accident,accident,node,17399.134735
1,1,death,death,node,23440.786330
2,2,disease,disease,node,26235.653549
3,3,pneumonium,pneumonium,node,6641.914576
4,4,cancer,cancer,node,20368.378862


In [ ]:
xr.head()

,start,end,effect,type
0,0,1,negative,relationship
1,0,10,negative,relationship
2,0,89,negative,relationship
3,0,53,negative,relationship
4,0,55,negative,relationship


In [ ]:
ndt = xd['id'].tolist()
eng = xr.groupby(['end'])
sng = xr.groupby(['start'])

In [ ]:
rank = xd['ranks'].tolist()

In [ ]:
d = 0.85
n = len(xd)

In [ ]:
im = (1-d)/n

In [ ]:
ranks = []

In [ ]:
for x in tqdm(ndt):
  r = 0
  sum = 0
  if x in eng.groups.keys():
    df = eng.get_group(x)
    ic = df['start'].tolist()
    for y in ic:
      if y in sng.groups.keys():
        dm = sng.get_group(y)
        l = len(dm)
        p = rank[y]/abs(l)
        sum+=p
  r = im + d * sum
  ranks.append(r)

100%|██████████| 337682/337682 [09:17<00:00, 605.35it/s] 


In [ ]:
xd['final_ranks'] = ranks

In [ ]:
xd.head()

,id,label,concept,type,ranks,final_ranks
0,0,accident,accident,node,17399.134735,1552.897128
1,1,death,death,node,23440.786330,11702.120120
2,2,disease,disease,node,26235.653549,5132.346620
3,3,pneumonium,pneumonium,node,6641.914576,1967.091368
4,4,cancer,cancer,node,20368.378862,3051.568576


In [ ]:
xd.tail()

,id,label,concept,type,ranks,final_ranks
337677,337677,employer to campaign year round,employer to campaign year round,node,49.356100,4.442049e-07
337678,337678,collapsing of the skull of the partially born ...,collapsing of the skull of the partially born ...,node,0.000000,4.442049e-07
337679,337679,risk of damage to the woman,risk of damage to the woman,node,148.068301,4.442049e-07
337680,337680,random sobriety stop,random sobriety stop,node,0.000000,4.442049e-07
337681,337681,unjust detection of other crime,unjust detection of other crime,node,296.136602,4.442049e-07


In [ ]:
L1 = [{k: v for k, v in x.items() if pd.notnull(v)} for x in xd.to_dict('r')]
L2 = [{k: v for k, v in x.items() if pd.notnull(v)} for x in xr.to_dict('r')]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  


In [ ]:
with open('./ReNewEfArgRanksFinal-Graph.json', 'w') as file:
    json.dump({ "Graph": L1, "Relationship": L2}, file)

In [ ]:
xd.to_csv('./FinalRankedGraph.csv')

# **Real Page Rank algorithm with Pos & Neg**

In [47]:
f = open('./ReNewEfArgRanksFinal-Graph.json',)

In [48]:
data = json.load(f)

In [49]:
f.close()

In [50]:
nodes = []
for i in tqdm(data['Graph']):
    px = []
    px.append(int(i['id']))
    px.append(i['label'])
    px.append(i['concept'])
    px.append(i['type'])
    px.append(i['ranks'])
    px.append(i['final_ranks'])
    nodes.append(px)
  
# Closing file
f.close()

100%|██████████| 337682/337682 [00:01<00:00, 177180.02it/s]


In [51]:
rels = []
for i in tqdm(data['Relationship']):
  px = []
  px.append(int(i['start']))
  px.append(int(i['end']))
  px.append(i['effect'])
  px.append(i['type'])
  rels.append(px)
  
# Closing file
f.close()

100%|██████████| 2033004/2033004 [00:04<00:00, 486151.15it/s]


In [53]:
xd = pd.DataFrame(
            nodes, columns=['id', 'label', 'concept', 'type', 'ranks', 'final_ranks'])
xr = pd.DataFrame(
            rels, columns=['start', 'end', 'effect', 'type'])

In [54]:
rnk = xd['final_ranks'].tolist()

In [55]:
f = open('./ReNewEfArgRanksPN-Graph.json',)

In [56]:
data = json.load(f)

In [57]:
f.close()

In [58]:
nodes = []
for i in tqdm(data['Graph']):
    px = []
    px.append(int(i['id']))
    px.append(i['label'])
    px.append(i['concept'])
    px.append(i['type'])
    px.append(i['ranks'])
    px.append(i['Pos'])
    px.append(i['Neg'])
    nodes.append(px)
  
# Closing file
f.close()

100%|██████████| 337682/337682 [00:00<00:00, 463553.52it/s]


In [59]:
rels = []
for i in tqdm(data['Relationship']):
  px = []
  px.append(int(i['start']))
  px.append(int(i['end']))
  px.append(i['effect'])
  px.append(i['type'])
  rels.append(px)
  
# Closing file
f.close()

100%|██████████| 2033004/2033004 [00:05<00:00, 362873.49it/s]


In [60]:
xd = pd.DataFrame(
            nodes, columns=['id', 'label', 'concept', 'type', 'ranks', 'Pos', 'Neg'])
xr = pd.DataFrame(
            rels, columns=['start', 'end', 'effect', 'type'])

In [61]:
xd['final_ranks'] = rnk

In [62]:
xd.head()

,id,label,concept,type,ranks,Pos,Neg,final_ranks
0,0,accident,accident,node,17399.134735,0.000000,17508.337566,1552.897128
1,1,death,death,node,23440.786330,13.324650,23751.858104,11702.120120
2,2,disease,disease,node,26235.653549,2072.677657,24777.582845,5132.346620
3,3,pneumonium,pneumonium,node,6641.914576,273.826409,6487.896418,1967.091368
4,4,cancer,cancer,node,20368.378862,0.000000,20963.631060,3051.568576


In [63]:
xr.head()

,start,end,effect,type
0,0,1,negative,relationship
1,0,10,negative,relationship
2,0,89,negative,relationship
3,0,53,negative,relationship
4,0,55,negative,relationship


In [64]:
orr = xr.groupby(['effect'])

In [65]:
pos = orr.get_group('positive')
neg = orr.get_group('negative')

In [66]:
len(pos)

331787

In [67]:
len(neg)

1701217

In [68]:
ndt = xd['id'].tolist()
eng = pos.groupby(['end'])
sng = pos.groupby(['start'])

In [69]:
rank = xd['Pos'].tolist()

In [70]:
d = 0.85
n = len(xd)

In [71]:
im = (1-d)/n

In [72]:
ranks = []

In [73]:
for x in tqdm(ndt):
  r = 0
  sum = 0
  if x in eng.groups.keys():
    df = eng.get_group(x)
    ic = df['start'].tolist()
    for y in ic:
      if y in sng.groups.keys():
        dm = sng.get_group(y)
        l = len(dm)
        p = rank[y]/abs(l)
        sum+=p
  r = im + d * sum
  ranks.append(r)

100%|██████████| 337682/337682 [01:20<00:00, 4204.66it/s]


In [74]:
xd['pRank'] = ranks

In [75]:
xd.head()

,id,label,concept,type,ranks,Pos,Neg,final_ranks,pRank
0,0,accident,accident,node,17399.134735,0.000000,17508.337566,1552.897128,4.442049e-07
1,1,death,death,node,23440.786330,13.324650,23751.858104,11702.120120,4.442049e-07
2,2,disease,disease,node,26235.653549,2072.677657,24777.582845,5132.346620,7.311114e+02
3,3,pneumonium,pneumonium,node,6641.914576,273.826409,6487.896418,1967.091368,9.672191e+02
4,4,cancer,cancer,node,20368.378862,0.000000,20963.631060,3051.568576,4.442049e-07


In [76]:
xd.tail()

,id,label,concept,type,ranks,Pos,Neg,final_ranks,pRank
337677,337677,employer to campaign year round,employer to campaign year round,node,49.356100,49.356100,0.000000,4.442049e-07,4.442049e-07
337678,337678,collapsing of the skull of the partially born ...,collapsing of the skull of the partially born ...,node,0.000000,0.000000,0.000000,4.442049e-07,4.442049e-07
337679,337679,risk of damage to the woman,risk of damage to the woman,node,148.068301,0.000000,148.068301,4.442049e-07,4.442049e-07
337680,337680,random sobriety stop,random sobriety stop,node,0.000000,0.000000,0.000000,4.442049e-07,4.442049e-07
337681,337681,unjust detection of other crime,unjust detection of other crime,node,296.136602,296.136602,0.000000,4.442049e-07,4.442049e-07


In [77]:
ndt = xd['id'].tolist()
eng = neg.groupby(['end'])
sng = neg.groupby(['start'])

In [78]:
rank = xd['Neg'].tolist()

In [79]:
d = 0.85
n = len(xd)

In [80]:
im = (1-d)/n

In [81]:
ranks = []

In [82]:
for x in tqdm(ndt):
  r = 0
  sum = 0
  if x in eng.groups.keys():
    df = eng.get_group(x)
    ic = df['start'].tolist()
    for y in ic:
      if y in sng.groups.keys():
        dm = sng.get_group(y)
        l = len(dm)
        p = rank[y]/abs(l)
        sum+=p
  r = im + d * sum
  ranks.append(r)

100%|██████████| 337682/337682 [06:05<00:00, 923.30it/s] 


In [83]:
xd['nRank'] = ranks

In [84]:
xd.head()

,id,label,concept,type,ranks,Pos,Neg,final_ranks,pRank,nRank
0,0,accident,accident,node,17399.134735,0.000000,17508.337566,1552.897128,4.442049e-07,1584.633381
1,1,death,death,node,23440.786330,13.324650,23751.858104,11702.120120,4.442049e-07,11357.284706
2,2,disease,disease,node,26235.653549,2072.677657,24777.582845,5132.346620,7.311114e+02,5160.155460
3,3,pneumonium,pneumonium,node,6641.914576,273.826409,6487.896418,1967.091368,9.672191e+02,1484.111504
4,4,cancer,cancer,node,20368.378862,0.000000,20963.631060,3051.568576,4.442049e-07,3088.935625


In [85]:
xd.tail()

,id,label,concept,type,ranks,Pos,Neg,final_ranks,pRank,nRank
337677,337677,employer to campaign year round,employer to campaign year round,node,49.356100,49.356100,0.000000,4.442049e-07,4.442049e-07,4.442049e-07
337678,337678,collapsing of the skull of the partially born ...,collapsing of the skull of the partially born ...,node,0.000000,0.000000,0.000000,4.442049e-07,4.442049e-07,4.442049e-07
337679,337679,risk of damage to the woman,risk of damage to the woman,node,148.068301,0.000000,148.068301,4.442049e-07,4.442049e-07,4.442049e-07
337680,337680,random sobriety stop,random sobriety stop,node,0.000000,0.000000,0.000000,4.442049e-07,4.442049e-07,4.442049e-07
337681,337681,unjust detection of other crime,unjust detection of other crime,node,296.136602,296.136602,0.000000,4.442049e-07,4.442049e-07,4.442049e-07


In [86]:
L1 = [{k: v for k, v in x.items() if pd.notnull(v)} for x in xd.to_dict('r')]
L2 = [{k: v for k, v in x.items() if pd.notnull(v)} for x in xr.to_dict('r')]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  


In [87]:
with open('./ReNewEfArgRanksFinalPN-Graph.json', 'w') as file:
    json.dump({ "Graph": L1, "Relationship": L2}, file)

In [88]:
xd.to_csv('./FinalRankedGraphPN.csv')